<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonTasks/blob/master/FactoryOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Распределительная задача. 

Завод может производить $Y$ видов продукции. На продукцию есть покупатель, которому требуется определенное количество каждого вида: $m_i$ 

Для каждого вида продукции он либо покупает требуемое количество, либо не покупает совсем, если мы произвели недостаточно. При этом известна цена контракта на каждый вид продукции: $c_i$ 

Завод одновременно может выпускать только один вид продукции со скоростью: $p_i$

Требуется написать математическую модель и найти оптимальный план производства на $S$ суток, если известно, что переход от одной продукции к другой происходит мгновенно, а его стоимость $R$ рублей.


Запишем математическую модель.

Введем переменные:

$ x_{i} = \left\{
\begin{array}{ll}
          1, & \mbox { если выполняем контракт на $i$ вид продукции,}\\
          0, & \mbox { в противном случае. } \\
\end{array} \right. $

Договоримся о входных данных:

$Y$ - множество видов продукции;

$m_i$ - необходимый объем $i$ продукции для выполнения контракта;

$p_i$ - скорость производства $i$ продукции;

$c_i$ - стоимость контракта на $i$ продукцию.

Математическая модель:

Целевая функция:

$$\sum\limits_{i = 0}^{I} c_i x_i - R((\sum\limits_{i = 0}^{I} x_i)-1) \to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i \in I $$

$$\sum\limits_{i = 0}^{I} (m_i /p_i) x_i \leqslant S $$

In [35]:
!pip install pulp
!pip install cplex

In [36]:
import numpy as np
import time
from random import randrange
import pulp as plp

In [48]:
Y = 10 
R = 49 
S = 17
m = [51, 42, 67, 25, 54, 20, 31, 57, 52, 52]
p = [27, 19, 23, 24, 23, 25, 19, 16, 15, 27]
c = [412, 405, 436, 677, 643, 686, 685, 602, 525, 458]

In [49]:
problem = plp.LpProblem(name='FactoryProblem', sense=plp.LpMaximize)
x  = {i : plp.LpVariable(cat=plp.LpBinary, name="x"+str(i)) for i in range(Y)}

problem.addConstraint(plp.LpConstraint(
                     e=plp.lpSum((m[i]/p[i]) * x[i] for i in range(Y)),
                     sense=plp.LpConstraintLE,
                     rhs=S ,
                     name="constraint_{1}"))

problem.setObjective(plp.lpSum(c[i] * x[i] for i in range(Y))-R*(plp.lpSum(x[i] for i in range(Y))-1))
print(problem)

FactoryProblem:
MAXIMIZE
363*x0 + 356*x1 + 387*x2 + 628*x3 + 594*x4 + 637*x5 + 636*x6 + 553*x7 + 476*x8 + 409*x9 + 49
SUBJECT TO
constraint_{1}: 1.88888888889 x0 + 2.21052631579 x1 + 2.91304347826 x2
 + 1.04166666667 x3 + 2.34782608696 x4 + 0.8 x5 + 1.63157894737 x6 + 3.5625 x7
 + 3.46666666667 x8 + 1.92592592593 x9 <= 17

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer
0 <= x5 <= 1 Integer
0 <= x6 <= 1 Integer
0 <= x7 <= 1 Integer
0 <= x8 <= 1 Integer
0 <= x9 <= 1 Integer



In [50]:
solver_list = plp.list_solvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'CPLEX_DLL', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'PULP_CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD']


In [51]:
def experiment(name, solver):
    start = time.time()
    problem.solve(solver)
    answer = plp.value(problem.objective)
    print(f"{name} solved {time.time() - start}  answer : {answer}")
    
#experiment('Gurobi', plp.get_solver('GUROBI', msg=False))
experiment('CPLEX', plp.get_solver('CPLEX_PY', msg=False))
#experiment('GLPK', plp.get_solver('GLPK_CMD', path="C:\GRB\winglpk-4.65\glpk-4.65\w64\glpsol.exe", msg=False))
experiment('PULP', plp.get_solver('PULP_CBC_CMD', msg=False))

CPLEX solved 0.020369768142700195  answer : 4345.0
PULP solved 0.025779247283935547  answer : 4345.0


In [52]:
for v in problem.variables():
            print(v.name, "=", v.varValue)

x0 = 1.0
x1 = 0.0
x2 = 0.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 1.0
x7 = 1.0
x8 = 1.0
x9 = 1.0


In [53]:
def experiment(name, solver, timeLimit):
    times = []
    start = time.time()
    current = start
    while current - start <= timeLimit:
        problem.solve(solver)
        startIter, current = current, time.time()
        times.append(current - startIter)
    
    answer = plp.value(problem.objective)
    print(f"{name} solved {len(times) - 1} instances "
          f"in {timeLimit} seconds answer : {answer}")

    
#experiment('Gurobi', plp.get_solver('GUROBI', msg=False), 120)
experiment('CPLEX', plp.get_solver('CPLEX_PY', msg=False), 120)
#experiment('GLPK', plp.get_solver('GLPK_CMD', path="C:\GRB\winglpk-4.65\glpk-4.65\w64\glpsol.exe", msg=False), 120)
experiment('PULP', plp.get_solver('PULP_CBC_CMD', msg=False), 120)

CPLEX solved 12979 instances in 120 seconds answer : 4345.0
PULP solved 6687 instances in 120 seconds answer : 4345.0
